In [ ]:
import numpy as np
samples = np.load('data/samples_300_meteo_ex_norm.npy')
# Extract features and target
X = samples[0:2,:].T
y = samples[2,:]

In [ ]:
# Split data into training and validation sets
# Convert data to PyTorch tensors
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
import torch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Ensure correct shape
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

In [23]:
from model import ClimateDataset
from torch.utils.data import DataLoader
# Create DataLoaders
batch_size = 64  # You can adjust this value based on your available memory and GPU capacity
train_loader = DataLoader(ClimateDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(ClimateDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

In [ ]:
from model import NeuralNet
from torch import nn, optim
# Initialize model, loss function, and optimizer
input_dim = 2  # Number of predictors (ssrd and vpd)
model = NeuralNet(input_dim)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    # Validation Step
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_val_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.10f}, "
            f"Val Loss: {avg_val_loss:.10f}")
            

Epoch [1/10], Train Loss: 0.0571664112, Val Loss: 0.0191733968
Epoch [2/10], Train Loss: 0.0209155047, Val Loss: 0.0185892272
Epoch [3/10], Train Loss: 0.0204728556, Val Loss: 0.0195243208
Epoch [4/10], Train Loss: 0.0202921504, Val Loss: 0.0188722982
Epoch [5/10], Train Loss: 0.0203867951, Val Loss: 0.0190267540
Epoch [6/10], Train Loss: 0.0202096698, Val Loss: 0.0187897348
Epoch [7/10], Train Loss: 0.0199748461, Val Loss: 0.0191293691
Epoch [8/10], Train Loss: 0.0200869272, Val Loss: 0.0188953649
Epoch [9/10], Train Loss: 0.0199710939, Val Loss: 0.0197343311
Epoch [10/10], Train Loss: 0.0198432181, Val Loss: 0.0193576552


In [ ]:
# Save the model weights
torch.save(model.state_dict(), "./outputs/model_weights_default_meteo_ex.pth")